In [6]:
!git clone https://github.com/Leejunho123/deep-learning-from-scratch-3.git
%cd /content/deep-learning-from-scratch-3

Cloning into 'deep-learning-from-scratch-3'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 2101 (delta 0), reused 0 (delta 0), pack-reused 2097
Receiving objects: 100% (2101/2101), 32.30 MiB | 40.48 MiB/s, done.
Resolving deltas: 100% (1444/1444), done.
/content/deep-learning-from-scratch-3


## 53 모델 저장 및 읽어오기


### 53.1 넘파이의 save 함수와 load함수

In [1]:
import numpy as np
x = np.array([1,2,3])
np.save('test.npy',x)

x = np.load('test.npy')
print(x)

[1 2 3]


In [2]:
x1 = np.array([1,2,3])
x2 = np.array([4,5,6])

np.savez('test.npz', x1=x1, x2=x2)
arrays = np.load('test.npz')

x1 = arrays['x1']
x2 = arrays['x2']

print(x1)
print(x2)

[1 2 3]
[4 5 6]


In [4]:
x1 = np.array([1,2,3])
x2 = np.array([4,5,6])
data = {'x1':x1, 'x2':x2}

np.savez('test.npz', **data)
arrays = np.load('test.npz')

x1 = arrays['x1']
x2 = arrays['x2']

print(x1)
print(x2)

[1 2 3]
[4 5 6]


### 53.2 Layer 클래스의 매개변수를 평평하게


In [10]:
import dezero
from dezero.layers import Layer, Parameter

layer = Layer()
l1 = Layer()
l1.p1 = Parameter(np.array(1))

layer.l1 = l1
layer.p2 = Parameter(np.array(2))
layer.p3 = Parameter(np.array(3))

params_dict = {}
layer._flatten_params(params_dict)
print(params_dict)


{'p2': variable(2), 'p3': variable(3), 'l1/p1': variable(1)}


In [11]:
class Layer :


  def _flatten_params(self, params_dict, parent_key = ''):
    for name in self._params:
      obj = self.__dict__[name]
      key = parent_key + '/' + name if parent_key else name

      if isinstance(obj, Layer):
        obj._flatten_params(params_dict, key)

      else: 
        params_dict[key] = obj

### 53.3 Layer 클래스의 save 함수와 load 함수

In [ ]:
import os

class Layer:

  def save_weights(self, path):
    self.to_cpu()
    params_dict = {}
    self._flatten_params(params_dict)
    array_dict = {key: param.data for key, param in params_dict.items() if param is not None}
    try : 
      np.savez_compressed(path, **array_dict)

    except (Exception, KeyboardInterrupt) as e:
      if os.path.exists(path):
        os.remove(path)
      raise

  def load_weights(self,path):
    npz = np.load(path)
    params_dict = {}
    self._flatten_params(params_dict)
    for key, param in params_dict.items():
      param.data = npz[key]

      
            

In [14]:
import os
import dezero
import dezero.functions as F
from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP

max_epoch = 3
batch_size = 100
train_set = dezero.datasets.MNIST(train = True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000,10))
optimizer = optimizers.SGD().setup(model)

if os.path.exists('my_mlp.npz'):
  model.load_weights('my_mlp.npz')

for epoch in range(max_epoch):
  sum_loss = 0

  for x, t in train_loader:
    y = model(x)
    loss = F.softmax_cross_entropy(y, t)
    model.cleargrads()
    loss.backward()
    optimizer.update()
    sum_loss += float(loss.data) * len(t)

  print('epoch: {}, loss: {:.4f}'.format(epoch+1,sum_loss/len(train_set)))

model.save_weights('my_mlp.npz')

Downloading: train-images-idx3-ubyte.gz


HTTPError: ignored